# JUICE RPWI HF Geometry -- 2024/10/6

In [ ]:
import csv
import datetime
import math
import matplotlib.pyplot as plt
import numpy as np
import sys

sys.path.append("./lib/")
import juice_spice_lib as juice_spice

### parameter set

In [ ]:
# Geometry data: http://spice.esac.esa.int/webgeocalc/#StateVector
ID_target   = 0             # 0:Sun  3:Earth  5:Jupiter  52:Europa  53:Ganymede  54:Callisto  99:JUICE
ID_observer = 99            # 0:Sun  3:Earth  5:Jupiter  52:Europa  53:Ganymede  54:Callisto  99:JUICE
ID_frame    = 99            # 0:Sun  3:Earth  5:Jupiter  52:Europa  53:Ganymede  54:Callisto  99:JUICE  991:RWI

# Plot interval
Epoch_min = '2024-08-19 00:00:00';  t_min0 = datetime.datetime.strptime(Epoch_min, "%Y-%m-%d %H:%M:%S")
Epoch_max = '2024-08-24 00:00:00';  t_max0 = datetime.datetime.strptime(Epoch_max, "%Y-%m-%d %H:%M:%S")
t_min0 = 0;  t_max0 = 0     # no-assign -- all duration

dump_mode = 1                                           # 0: no-dump  1:plot dump
work_dir  = '/Users/user/0-python/JUICE_python/ql/'     # Plot dump folder

### Geometry File Read

In [ ]:
name_dir = './mk/' #/Users/user/D-Univ/data/data-JUICE/geometry/'
name_file, name_target, name_observer, name_frame = juice_spice.name_geometry(ID_target, ID_observer, ID_frame, Epoch_min, Epoch_max)
name_geometry_file = name_dir + name_file
print(name_geometry_file)

In [ ]:
with open(name_geometry_file, 'r') as f:
    reader = csv.reader(f);  i = 1
    list = [row for row in reader]

In [ ]:
n_num = len(list)
j = 0;  k = 0
for i in range(n_num):
    if len(list[i]) < 10 and i < 17:
        j = i
    if len(list[i]) < 10 and i > 17:
        k = i
        break
list = list[j+1:k];  n_num = len(list)

In [ ]:
class struct:
    pass

geometry = struct();  n_num = len(list)
geometry.distance  = np.zeros((   n_num-1))
geometry.location  = np.zeros((3, n_num-1))
geometry.direction = np.zeros((3, n_num-1))
geometry.epoch     = []

for i in range(n_num):
    if i>0:
        time = datetime.datetime.strptime(list[i][0][0:19], '%Y-%m-%d %H:%M:%S')
        geometry.epoch.append(time)
        geometry.location [0][i-1] = list[i][3]; geometry.location[1][i-1] = list[i][4]; geometry.location[2][i-1] = list[i][5]
        geometry.distance [i-1] = (geometry.location[0][i-1]**2 + geometry.location[1][i-1]**2 + geometry.location[2][i-1]**2)**0.5
geometry.direction = geometry.location / geometry.distance

distance_max = np.nanmax(geometry.distance)

In [ ]:
# Time, Direction[km], Distance[km]
print("[Target]", name_target, "  [Observer]", name_observer, "  [Frame]", name_frame,  '  [Total]', n_num)
print(' [Start]', geometry.epoch[0],  f'  Direction ({geometry.direction[0][ 0]:+.3f}, {geometry.direction[1][ 0]:+.3f}, {geometry.direction[2][ 0]:+.3f})', f' Distance[km] {geometry.distance[ 0]:.3e}',  f'({geometry.location[0][ 0]:+.3e}, {geometry.location[1][ 0]:+.3e}, {geometry.location[2][ 0]:+.3e})')
print('   [End]', geometry.epoch[-1], f'  Direction ({geometry.direction[0][-1]:+.3f}, {geometry.direction[1][-1]:+.3f}, {geometry.direction[2][-1]:+.3f})', f' Distance[km] {geometry.distance[-1]:.3e}',  f'({geometry.location[0][-1]:+.3e}, {geometry.location[1][-1]:+.3e}, {geometry.location[2][-1]:+.3e})')

### Plot

In [ ]:
fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(2, 1, 1);  ax2 = fig.add_subplot(2, 1, 2)

ax1.plot(geometry.epoch, geometry.direction[0], '-r', linewidth=1, label='X')
ax1.plot(geometry.epoch, geometry.direction[1], '-g', linewidth=1, label='Y')
ax1.plot(geometry.epoch, geometry.direction[2], '-b', linewidth=1, label='Z')
ax2.plot(geometry.epoch, geometry.distance,     '-y', linewidth=3, label='R')
ax2.plot(geometry.epoch, geometry.location[0],  '-r', linewidth=1, label='X')
ax2.plot(geometry.epoch, geometry.location[1],  '-g', linewidth=1, label='Y')
ax2.plot(geometry.epoch, geometry.location[2],  '-b', linewidth=1, label='Z')
ax1.set_ylabel('Direction');  ax2.set_ylabel('Distance [km]')
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8)
title_label = '[direction & distance of ' + name_target + ' from ' + name_observer + ' in ' + name_frame + ' frame]';  ax1.set_title(title_label)

if t_min0 == 0:
    E_min = '2024-08-23 03:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
    E_max = '2024-08-23 06:00:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
    xlim=[geometry.epoch[0], geometry.epoch[-1]]
else:
    xlim=[t_min0, t_max0]
print("==> [X-axis] ", xlim[0].strftime('%Y%m%d %H%M -'), xlim[1].strftime('%Y%m%d %H%M'));  ax1.set_xlim(xlim);  ax2.set_xlim(xlim)
ylim=[-1, +1];                      ax1.set_ylim(ylim)
ylim=[-distance_max, distance_max]; ax2.set_ylim(ylim)

fig.show
if dump_mode == 1:
    png_filename = work_dir + 'geometry-' + name_target + '-' + name_observer + '-' + name_frame + '_' + xlim[0].strftime('%Y%m%d%H%M-') + xlim[1].strftime('%Y%m%d%H%M') + '.png'
    fig.savefig(png_filename);  print(png_filename)

In [ ]:
if ID_frame == 99 or ID_frame == 991:
    if ID_frame == 99:
        geometry.direction_sc     = geometry.direction
        geometry.direction_rwi    = np.zeros((3, n_num-1))
        geometry.direction_rwi[0] = geometry.direction_sc[0] *(-0.87494143) + geometry.direction_sc[1] *(+0.33033414) + geometry.direction_sc[2] *(-0.35405770)
        geometry.direction_rwi[1] = geometry.direction_sc[0] *(+0.26827433) + geometry.direction_sc[1] *(-0.27793454) + geometry.direction_sc[2] *(-0.92237806)
        geometry.direction_rwi[2] = geometry.direction_sc[0] *(-0.40310650) + geometry.direction_sc[1] *(-0.90200559) + geometry.direction_sc[2] *(+0.15456734)
    if ID_frame == 991:
        geometry.direction_rwi    = geometry.direction
        geometry.direction_sc     = np.zeros((3, n_num-1))
        geometry.direction_sc[0]  = geometry.direction_rwi[0]*(-0.87494143) + geometry.direction_rwi[1]*(+0.26827433) + geometry.direction_rwi[2]*(-0.40310650)
        geometry.direction_sc[1]  = geometry.direction_rwi[0]*(+0.33033414) + geometry.direction_rwi[1]*(-0.27793454) + geometry.direction_rwi[2]*(-0.90200559)
        geometry.direction_sc[2]  = geometry.direction_rwi[0]*(-0.3540577)  + geometry.direction_rwi[1]*(-0.92237806) + geometry.direction_rwi[2]*(+0.15456734)
    
    print("[Target]", name_target, "  [Observer]", name_observer, "  [Frame]", name_frame,  '  [Total]', n_num)
    print(' [Start]', geometry.epoch[ 0],  f'  Direction_SC ({geometry.direction_sc[0][ 0]:+.3f}, {geometry.direction_sc[1][ 0]:+.3f}, {geometry.direction_sc[2][ 0]:+.3f})', f'  Direction_RWI ({geometry.direction_rwi[0][ 0]:+.3f}, {geometry.direction_rwi[1][ 0]:+.3f}, {geometry.direction_rwi[2][ 0]:+.3f})')
    print('   [End]', geometry.epoch[-1],  f'  Direction_SC ({geometry.direction_sc[0][-1]:+.3f}, {geometry.direction_sc[1][-1]:+.3f}, {geometry.direction_sc[2][-1]:+.3f})', f'  Direction_RWI ({geometry.direction_rwi[0][-1]:+.3f}, {geometry.direction_rwi[1][-1]:+.3f}, {geometry.direction_rwi[2][-1]:+.3f})')

In [ ]:
if ID_frame == 99 or ID_frame == 991:
    fig = plt.figure(figsize=(14, 11))
    ax1 = fig.add_subplot(2, 1, 1);  ax2 = fig.add_subplot(2, 1, 2)

    ax1.plot(geometry.epoch, geometry.direction_sc[0],  '-r', linewidth=1, label='X')
    ax1.plot(geometry.epoch, geometry.direction_sc[1],  '-g', linewidth=1, label='Y')
    ax1.plot(geometry.epoch, geometry.direction_sc[2],  '-b', linewidth=1, label='Z')
    ax2.plot(geometry.epoch, geometry.direction_rwi[0], '-r', linewidth=1, label='X')
    ax2.plot(geometry.epoch, geometry.direction_rwi[1], '-g', linewidth=1, label='Y')
    ax2.plot(geometry.epoch, geometry.direction_rwi[2], '-b', linewidth=1, label='Z')
    ax1.set_ylabel('Direction SC-frame');  ax2.set_ylabel('Direction RWI-frame')
    ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8)
    title_label = '[direction of ' + name_target + ' from ' + name_observer + ' in ' + name_frame + ' frame]';  ax1.set_title(title_label)

    if t_min0 == 0:
        E_min = '2024-08-23 03:00:00';  t_min = datetime.datetime.strptime(E_min, "%Y-%m-%d %H:%M:%S");  
        E_max = '2024-08-23 06:00:00';  t_max = datetime.datetime.strptime(E_max, "%Y-%m-%d %H:%M:%S");  xlim=[t_min, t_max]
        xlim=[geometry.epoch[0], geometry.epoch[-1]]
    else:
        xlim=[t_min0, t_max0]
    print("==> [X-axis] ", xlim[0].strftime('%Y%m%d %H%M -'), xlim[1].strftime('%Y%m%d %H%M'));  ax1.set_xlim(xlim);  ax2.set_xlim(xlim)
    ylim=[-1, +1];  ax1.set_ylim(ylim);  ax2.set_ylim(ylim)

    fig.show
    if dump_mode == 1:
        png_filename = work_dir + 'geometry2-' + name_target + '-' + name_observer + '-' + name_frame + '_' + xlim[0].strftime('%Y%m%d%H%M-') + xlim[1].strftime('%Y%m%d%H%M') + '.png'
        fig.savefig(png_filename);  print(png_filename)